In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os

In [ ]:
PATH =  os.getcwd()

In [ ]:
%run -i code/utils.py

# 1. Initialization

In [ ]:
# Prompts user for FDS and fds2ascii executable paths & email to send notifcation
#fds_bin = input('File path to FDS executable: ')
#fds2ascii = input('File path to fds2ascii utility in FDS: ')
fds_bin = '/home1/02410/lvalera/fds/Build/impi_intel_linux_64/fds_impi_intel_linux_64'          # Stampede2/FRONTERA
fds2ascii = '/home1/02410/lvalera/FDS/FDS6/bin/fds2ascii'               # Stampede2/FRONTERA

#fds_bin = '/home/leobardovalera/FDS/FDS6/bin/fds'             # Local
#fds2ascii = '/home/leobardovalera/FDS/FDS6/bin/fds2ascii'     # Local

## 1.1 Initial Region of Interest

In [ ]:
# Reads in elevation file to extract region information
name_of_file = 'data/Elevations_Files/gatlinburg_res_1x1.elv'
Mst_original = pd.read_csv(name_of_file )
Image_name   = 'data/Regions_Images/Gatlinburg_texted.png'

In [ ]:
# Global Coordinates
North = 1000
South = 0
East  = 1000
West  = 0
# Coordinates of the Region's Border 
Min_x = 500      
Max_x = 700

Min_y = 200
Max_y = 400

Mst   = Mst_original.copy()
Mst = Mst[(Mst.x >= Min_x) 
        & (Mst.x <= Max_x) 
        & (Mst.y >= Min_y) 
        & (Mst.y <= Max_y)]

In [ ]:
# Defines the Resolution of the Meshes and the obstacles (in Meters)
R = 2                                   # Resolution of Cells
Mst = Mst[(Mst.x%R==0) & (Mst.y%R==0)]  # Filters data 
Ro = 2    # Resolution of Obstacles

# Radius of the fire from the center
fire_radius   = 100

In [ ]:
Mst.shape

In [ ]:
# Defines range of elevation
Min_z = 380
Max_z = 620
Regiones = [Min_x,Max_x,Min_y,Max_y,Min_z,Max_z]

## 1.2 Input FDS File Parameters

In [ ]:
# Index for regions that we are simulating
num_region = 1

In [ ]:
# Set time interval
T_begin    = 0.0
DTT        = 60.0
T_end      = T_begin + DTT
DT         = 0.1

# Set Number of meshes in x, y, z directions
Nmx        = 2     
Nmy        = 2    
Nmz        = 4

PC         = 1         # Predictor-Corrector Status Flag
Location   = [608,260] # Initial Location of the fire
Child      = f"Region{num_region}" # Start of Sequential Domain Decomposition
HRRPUA     = 3500      # Heat Release Rate (HRR)
rampa_time = 10       # Time that the fire ramp lasts

UVW_Timer  = 0
DT_UVW_Timer = 60
UVW_Timer  =  UVW_Timer + DT_UVW_Timer      # Time


# File name conventions
foldername = f'chimney'
filename   = f'{Child}.fds'

# Heat Release Rate (HRR) Minimum
#hrr_threshold = Ro*Ro*Ro

# Defining the dataframe with the HRR information
Hrr = Mst[(Mst['x'] <= Location[0]+Ro) & (Mst['x'] >= Location[0]-Ro) & (Mst['y'] <= Location[1]+Ro) & (Mst['y'] >= Location[1]-Ro)]
Hrr = Hrr[[Hrr.columns[0],Hrr.columns[1]]]
Hrr['hrr'] = HRRPUA

if not path.exists(f"{PATH}/simulations/{foldername}"):
    os.mkdir(f"{PATH}/simulations/{foldername}")

if not path.exists(f"{PATH}/simulations/{foldername}/{Child}"):
    os.mkdir(f"{PATH}/simulations/{foldername}/{Child}")

Hrr.to_csv(f'{PATH}/simulations/{foldername}/{Child}/Hrr_{Child}.csv', index=False)
# Function to write input FDS file with parameter specified above

In [ ]:
write_fds_file(T_begin, T_end, DT, PC, Nmx, Nmy, Nmz, Hrr, Child)

## 1.3 Sets Folders

In [ ]:
FDS_FOLDER = f"simulations/{foldername}/{Child}" # Location of input FDS file

os.chdir(f'{PATH}/{FDS_FOLDER}')      # Changes directory to run the input FDS file location

## 1.4 Runs FDS

In [ ]:
# User decides to run as a job on a job scheduler or directly through mpiexec command
#job_type = input("Are you running with a job scheduler? \nIf so, is it LSF or Slurm?\nIf not, you do not have to specify. ")
job_type =''

In [ ]:
# Sets variables to use in job scripts/commands
num_nodes = 1             # Number of compute nodes to run job on
max_time = '3:00'         # Max time needed to run job (d-hh:mm:ss)
number_of_process = 16     # Number of processes(cores) needed to run the created meshes
omp_threads = 4           # Sets OpenMP Threads per process
jobName = f'FDS_{Child}'

In [ ]:
# Options: LSF, Slurm, or direct command (mpiexec)
if job_type=="LSF" or job_type=="Lsf" or job_type=='lsf':
    USER = os.getlogin()
    create_job_script_lsf(Child, num_nodes, max_time, omp_threads)    # Creates LSF job submission script
    os.system(f'bsub < job_{Child}.bsub') # Submits job script to the LSF job scheduler
    
    job_id = Get_job_id(['bjobs', '-u', USER])
    jobs = [job_id]
    wait_on_lsf()
    
elif job_type=="SLURM" or job_type=="Slurm" or job_type=="slurm":
    USER = os.getlogin()
    
#     partition = 'skx-normal' # Can be changed to another other partition used on TACC's Stampede2
    
    create_job_script_slurm(Child, num_nodes, max_time, omp_threads)  # Creates Slurm job submission script
    os.system(f'sbatch job_{Child}.sh')   # Submits job script to the Slurm job Scheduler
    
    job_id - Get_job_id(['squeue', '-u', USER])
    jobs = [job_id]
    wait_on_slurm()
    
else: 
    os.environ['OMP_NUM_THREADS'] = f'{omp_threads}'                  # Sets OpenMP Threads to 4
    os.system(f"mpiexec -n {number_of_process} {fds_bin} {filename}") # Runs FDS using 'mpiexec' command